# Этот колаб собирает базу из pdf файлов со старниц сайта.

In [ ]:
# инсталлируем библиотеку парсера
!pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=cc219d0a03615570b8b02fd8b30c24174e58f327699a742c4f953c3b7c764265
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [ ]:
# импортируем нужные библиотеки
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import os
from tika import parser

In [ ]:
# cкачиваем фалы данных в виде csv

def load_document_csv(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=tsv')
    response.raise_for_status()
    text = response.content

    return text

urls = load_document_csv('https://docs.google.com/spreadsheets/d/1h5BNBTXwzb8nMTIcre0vCDcujax1Fvfw3RdwT7FeAt8/edit?usp=sharing')
with open('urls_r.csv','wb') as f:
  f.write(urls)

# загружаем ссылки из файла данных в массим Numpy
links = np.genfromtxt('urls_r.csv', delimiter='\t', dtype=str, encoding='utf-8' )

In [ ]:
'''
Эта функция находит все файлы pdf на странице сайта.

Вход:
  url - адрес страницы сайта
Выход:
  title - заголовок страницы
  PDF_name - словарь в котором key - это название тега, value - ссылка на файл
'''

def find_pdf(url: str) -> str:

  # заводим переменную для названия страницы сайта
  title = ""
  # Заводим пустой словарь для списка pdf файлов
  PDF_name = {}

  # Получаем запрос от страницы
  response = requests.get(url)
  # Получаем ПрекрасныйСуп
  bs = BeautifulSoup(response.text, "html.parser")
  # сохраняем заголовок страницы
  title = bs.title.string

  # Определяем функцию отбора тегов. Берём теги у которых есть ссылка.
  def have_href(href):
    return href

  # Отбитаем теги у которых есть ссылка
  bs.find_all(href = have_href)


  # Наполняем словарь описаниями файла и ссылками
  # То, что это словарь - важно.
  # Так мы избегаем одинаковых файлов, но с разными ссылками. (такое есть)
  for tag in bs.find_all(href = have_href):
      # Извлекаем ссылку из тега
      tag_href = tag.attrs['href']
      # Делаем условие, если ссылка на файл, который оканчивается на .pdf
      if tag_href.split('.')[-1] == 'pdf' and \
        ("Обзор функций (PDF)" not in tag.get_text()): # это условие только для сайта KIA
        # Добавляем новый элемент в словарь
        PDF_name[tag.get_text().strip()] = tag_href
  # Возвращаем словарь
  return title, PDF_name

In [ ]:
'''
Эта функция принимает на вход URL-адрес
по которому находится pdf файл, переводит его в текстовый формат
при помощи tika и сохраняет на диск.

  Вход:
    url - адрес PDF файла
  Выход:
    topic - текст
'''

def pdf_to_txt_tika(url):
  topic = ''
  # pattern = re.compile(r'\w[а-я]')
  # получаем файл
  pdf_req = requests.get(url)
  # проверяем, что ссылка удачно парсится
  if pdf_req.status_code == 200:
    # запоминаем название файла без расширения
    f_name = os.path.basename(url).split('.')[0]
    # записываем файл на диск
    with open(f'{f_name}.pdf','wb') as f:
      f.write(pdf_req.content)

    # парсим pdf файл
    reader = parser.from_file(f'{f_name}.pdf')
    # сохраняем текст в переменную
    text = reader['content']
    # убираем спецсимволы
    text = re.sub(u"\uFFFD",' ', text)
    # убираем повторяющиеся пробелы внутри строки
    text = re.sub('\u0020+',' ', text)
    # убираем ошибочные разрывы абзацев
    text = text.replace(' \n\n', '\n')
    text = text.replace('. \n', '\n')
    text = text.replace(': \n', ':\n')
    text = text.replace('; \n', ';\n')
    text = text.replace(' \n', ' ')
    text = text.replace('-\n', '-')
    # разбиваем текст на строки для дальнейшей обработки
    txt_split = text.split('\n')

    # начинаем исправлять типичные ошибки парсера
    for st in txt_split:
      # убираем пробелы в начале строки
      st = st.lstrip()
      # проверям, что строка это не номер страницы
      num = re.match("^[0-9\s*]+$", st)
      # если строка не пустая и не номер страницы - добавляем её к тексту
      if not num and st:
        topic += st + '\n'

    topic = topic.replace(u' \u2022', u'\n\u2022')

    # снова разбиваем текст на строки для дальнейшей обработки
    txt_split = topic.split('\n')
    topic = ''

    for st in txt_split:
      if st:
        if re.match('[а-яёa-zA-Z\u00AB]', st[0]) and st:
          topic += ' ' + st
        else:
          topic += '\n' + st


    # и снова разбиваем текст, чтобы убрать повторы
    txt_split = topic.split('\n')
    topic = ''
    last_st = ''

    for st in txt_split:
      if last_st != st:
        topic +=  st + '\n'
      last_st = st


    # стираем pdf файл
    if os.path.isfile(f'{f_name}.pdf'):
      os.remove(f'{f_name}.pdf')

    # записываем текст в файл с названием оригинала, но txt
    with open(f'{f_name}.txt','w') as f:
      f.write(topic)

    return topic


Смотрим, как работает функция на примере одной станицы сайта:

In [ ]:
# Берём произвольную страницу из БЗ
url = 'https://www.kia.ru/auction/'

# Извлекаем "словарь" (список файлов pdf с сайта)
title, PDFs = find_pdf(url)

# Условие: Что-то печаем, если на странице сайта есть pdf файлы
if PDFs:
  # Если PDFки есть, то то печатаем заголовок
  print(f'Страница: {title}')
  print(f'Адрес страницы: {url}')

  # Распечатываем PDF файлы, которые есть на этой странице
  for key,value in PDFs.items():
    print(f'PDF - "{key}"\n\tссылка: {value}')

Страница: Kia Аукцион: продай свой автомобиль Kia с максимальной выгодой
Адрес страницы: https://www.kia.ru/auction/
PDF - "Правила проведения Аукциона для Продавцов"
	ссылка: https://cdn.kia.ru/media-data/landing/auction/docs/rules_for_individuals.pdf
PDF - "Акт приема-передачи автомобиля"
	ссылка: https://cdn.kia.ru/media-data/landing/auction/docs/act_of_acceptance_transfer_of_the_car.pdf
PDF - "Шаблон договора купли-продажи автомобиля"
	ссылка: https://cdn.kia.ru/media-data/landing/auction/docs/sale_contract_template.pdf


Собираем базу в файл base_PDF.

База сохраняется в корневой директории, там же все pdf файлы по отдельности.

In [ ]:
# переменная для подсчёта обработанных страниц
count_pages = 0
# переменная для подсчёта найденных pdf файлов
count_pdfs = 0
# переменная базы
base_PDF = ''

# Идём по списку ссылок
for row in links:
  count_pages += 1

  # Ищем все вхождения pdf файлы на странице
  title, PDFs = find_pdf(row[1])

  # Условие: Что-то печатаем, если pdf файлы на странице сайта есть
  if PDFs:
    # Если PDFки есть, то печатаем заголовок
    print(f'\nСтраница: {title}')
    print(f'Адрес страницы: {row[1]}')
    base_PDF += f'# {title}\n{title}\n'

    # Распечатываем PDF файлы, которые встречаются на страницах
    for key,value in PDFs.items():
      print(f'PDF - "{key}"\n\tссылка: {value}')
      count_pdfs += 1

      if key != "Скачать брошюру":
        base_PDF += f'## {key}\n{key}\n{pdf_to_txt_tika(value)}'
      else:
        base_PDF += f'{pdf_to_txt_tika(value)}'

# записываем базу в файл
with open(f'base_PDF.txt','w') as f:
  f.write(base_PDF)


# печатаем общий результат
print(f'\nВсего страниц: {count_pages}\nВсего файлов pdf: {count_pdfs}')


Страница: Kia Picanto 2022 – Киа Пиканто на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/picanto/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Picanto_Pre.pdf

Страница: Kia Rio 2022 – Седан Киа Рио на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/rio/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Rio_New.pdf

Страница: Kia Rio X 2022 – Кросс-хэтчбек Киа Рио Х на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/rio-x/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Rio_X_Preview_New.pdf

Страница: Kia Ceed 2022 – Хэтчбек Киа Сид на официальном сайте Kia в России
Адрес страницы: https://www.kia.ru/models/ceed/desc/
PDF - "Скачать брошюру"
	ссылка: https://cdn.kia.ru/master-data/brochures/Kia_Ceed_new_preview.pdf

Страница: Kia Ceed SW 2022 – Универсал Киа Сид СВ на официальном сайте Kia в России
Адре